## 5.3 렌즈 왜곡
### 5.3.1 리매핑
- dst = cv2.remap(src, mapx, mapy, interpolation, [, dst, borderMode, borderValue]) 
    - mapx, mapy : x, y축으로 이동할 좌표(인덱스), src와 동일한 크기 (dtype=float32)
    - 나머지는 cv2.warpAffine과 동일

In [5]:
import cv2
import numpy as np
import time

img = cv2.imread('img/girl.jpg')
rows , cols = img.shape[:2]

# 뒤집기 변환행렬로 구현
st1 = time.time()
mflip = np.float32([[-1, 0, cols-1], [0, -1, rows-1]])
fliped1 = cv2.warpAffine(img, mflip, (cols, rows))
print('matrix : ', time.time()-st1)

# remap 함수로 뒤집기 구현 
# (time이 직접 구현한 것에 비해서 조금 더 걸림)
# 비선형 변환(렌즈 왜곡과 관련한 보정이나 효과)에서 사용하는 것이 효과적
st2 = time.time()
mapy, mapx = np.indices((rows, cols), dtype=np.float32)
# indices 행렬 값을 초기화함, np.zeros보다는 속도가 빠름 (0이 되는 것이 아니므로)
mapx = cols - mapx - 1
mapy = rows - mapy - 1
fliped2 = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
print('remap : ', time.time() - st2)

# result
cv2.imshow('origin', img)
cv2.imshow('fliped1', fliped1)
cv2.imshow('fliped2', fliped2)
cv2.waitKey()
cv2.destroyAllWindows()

matrix :  0.002992391586303711
remap :  0.004987478256225586


In [6]:
import cv2
import numpy as np

l = 20 # wavelength
amp = 15 # 진폭

img = cv2.imread('img/taekwonv1.jpg')
rows , cols = img.shape[:2]

# 초기 매핑 배열 생성
mapy, mapx = np.indices((rows, cols), dtype=np.float32)

# indices 행렬 값을 초기화함, np.zeros보다는 속도가 빠름 (0이 되는 것이 아니므로)
sinx = mapx + amp*np.sin(mapy/l)
cosy = mapy + amp*np.cos(mapx/l)

# 영상 리매핑
img_sinx = cv2.remap(img, sinx, mapy, cv2.INTER_LINEAR) # x축만 sin 곡선 적용
img_cosy = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR) # y축만 cos 곡선 적용
# x, y 축 모두 sin, cos 적용 및 외곽 영역 보정
img_both = cv2.remap(img, sinx, cosy, cv2.INTER_LINEAR, None, cv2.BORDER_REPLICATE)

# result
cv2.imshow('origin', img)
cv2.imshow('sinx', img_sinx)
cv2.imshow('cosy', img_cosy)
cv2.imshow('sin & cos', img_both)
cv2.waitKey()
cv2.destroyAllWindows()

### 5.3.2 오목렌즈와 볼록 렌즈 왜곡
- 직교좌표계 : (x, y) 좌표
- 극좌표계 : (r, theta) 좌표
- 직교좌표 -> 극좌표 : r, theta = cv2.cartToPolar(mapx, mapy)
- 극좌표 -> 직교좌표 : mapx, mapy = cv2.polarToCart(r, theta)

In [11]:
import cv2
import numpy as np

img = cv2.imread('img/taekwonv1.jpg')
rows , cols = img.shape[:2]

# 설정값 세팅
exp = 1.5 # 볼록, 오목지수 (오목: 0.1~1, 볼록: 1.1~)
# exp = 0.5
scale = 1 # 변환 영역 크기 (0~1)

# 초기 매핑 배열 생성
mapy, mapx = np.indices((rows, cols), dtype=np.float32)

# 좌상단 기준좌표에서 -1~1로 정규화된 중심점 기준좌표로 변경
mapx = 2*mapx/(cols - 1) - 1
mapy = 2*mapy/(rows - 1) - 1

# 직교좌표를 극좌표로 변환
r, theta = cv2.cartToPolar(mapx, mapy)

# 왜곡 영역만 중심 확대/축소 지수 적용
r[r<scale] = r[r<scale]**exp # scale보다 작은 부분에 대해서 왜곡
# r = r**exp

# 극좌표를 직교좌표로 변환
mapx, mapy = cv2.polarToCart(r, theta)

# 중심점 기준에서 좌상단 기준으로 변경
mapx = ((mapx + 1)*cols - 1)/2
mapy = ((mapy + 1)*rows - 1)/2

# 리매핑 변환
distorted = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# result
cv2.imshow('origin', img)
cv2.imshow('distorted', distorted)
cv2.waitKey()
cv2.destroyAllWindows()

### 5.3.3 방사왜곡
- 렌즈 가장자리에 생기는 왜곡 현상을 해결하기 위한 방법
- 관련 수식 : rd = ru*(1 + k1*ru**2 + k2*ru**4 + k3*ru**6)
- 배럴 왜곡 : 밖으로 튀어나옴
- 핀쿠션 왜곡 : 안으로 들어감

In [14]:
import cv2
import numpy as np

# 왜곡계수 설정
# k1, k2, k3 = 0.5, 0.2, 0.0 # 배럴 왜곡
k1, k2, k3 = -0.3, 0.0, 0.0 # 핀쿠션 왜곡

img = cv2.imread('img/girl.jpg')
rows , cols = img.shape[:2]

# 초기 매핑 배열 생성
mapy, mapx = np.indices((rows, cols), dtype=np.float32)

# -1~1로 정규화된 중심점 기준좌표로 변경
mapx = 2*mapx/(cols - 1) - 1
mapy = 2*mapy/(rows - 1) - 1
r, theta = cv2.cartToPolar(mapx, mapy)

# 방사왜곡 변형연산
ru = r*(1 + k1*(r**2) + k2*(r**4) + k3*(r**6))

# 극좌표를 직교좌표로 변환
mapx, mapy = cv2.polarToCart(ru, theta)
mapx = ((mapx + 1)*cols - 1)/2
mapy = ((mapy + 1)*rows - 1)/2

# 리매핑 변환
distorted = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

# result
cv2.imshow('origin', img)
cv2.imshow('distorted', distorted)
cv2.waitKey()
cv2.destroyAllWindows()

### 렌즈의 왜곡을 제거할 목적으로 cv2.undistort() 함수 사용
- distorted = cv2.undistort(img, cameraMatrix, distCoeffs)

- cameraMatrix : [[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    - cx, cy : 내부 파라미터 중심점
    - fx, fy : 초점거리
    
- distCoeffs : 왜곡 계수, 최소 4개 or 5, 8, 12, 14개
    - (k1, k2, p1, p2[, k3])

In [17]:
import cv2
import numpy as np

# 격자무늬 생성
img = np.full((300, 400, 3), 255, np.uint8)
img[::10, :, :] = 0
img[:, ::10, :] = 0
width = img.shape[1]
height = img.shape[0]

# 왜곡계수 설정
# k1, k2, p1, p2 = 0.001, 0.0, 0.0, 0 # 배럴 왜곡: 0.0001부터 발생
k1, k2, p1, p2 = -0.0005, 0.0, 0.0, 0 # 핀쿠션 왜곡: -0.0001부터 발생
distCoeff = np.float64([k1, k2, p1, p2])

# 임의의 값으로 카메라 matrix 설정
fx, fy = 10, 10
cx, cy = width/2, height/2
camMtx = np.float32([[fx, 0, cx],
                    [0, fy, cy],
                    [0, 0, 1]])

# 왜곡 변형
distorted = cv2.undistort(img, camMtx, distCoeff)

# result
cv2.imshow('origin', img)
cv2.imshow('distorted', distorted)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 5.4 실전 예제
### 5.4.1 모자이크 처리1

In [1]:
import cv2
import numpy as np

rate = 15
win_title = 'mosaic'
img = cv2.imread('img/taekwonv1.jpg')

while True:
    x, y, w, h = cv2.selectROI(win_title, img, False)
    if w and h:
        roi = img[y:y+h, x:x+w] # 관심영역 지정
        roi = cv2.resize(roi, (w//rate, h//rate))
        # 기존 크기로 확대
        roi = cv2.resize(roi, (w, h), interpolation=cv2.INTER_AREA)
        img[y:y+h, x:x+w] = roi # 원본 이미지에 적용
        cv2.imshow(win_title, img)
    else:
        break

cv2.waitKey()
cv2.destroyAllWindows()

### 5.4.2 리퀴파이 도구

In [2]:
import cv2
import numpy as np

win_title = 'Liquify'   # 창 이름
half = 50               # 관심 영역 절반 크기
isDragging = False      # 드래그 여부 플래그

# 리퀴파이 함수
def liquify(img, cx1,cy1, cx2,cy2) :
    # 대상 영역 좌표와 크기 설정
    x, y, w, h = cx1-half, cy1-half, half*2, half*2
    # 관심 영역 설정
    roi = img[y:y+h, x:x+w].copy()
    out = roi.copy()

    # 관심영역 기준으로 좌표 재 설정
    offset_cx1,offset_cy1 = cx1-x, cy1-y
    offset_cx2,offset_cy2 = cx2-x, cy2-y
    
    # 변환 이전 4개의 삼각형 좌표
    tri1 = [[ (0,0), (w, 0), (offset_cx1, offset_cy1)], # 상,top
            [ [0,0], [0, h], [offset_cx1, offset_cy1]], # 좌,left
            [ [w, 0], [offset_cx1, offset_cy1], [w, h]], # 우, right
            [ [0, h], [offset_cx1, offset_cy1], [w, h]]] # 하, bottom

    # 변환 이후 4개의 삼각형 좌표
    tri2 = [[ [0,0], [w,0], [offset_cx2, offset_cy2]], # 상, top
            [ [0,0], [0, h], [offset_cx2, offset_cy2]], # 좌, left
            [ [w,0], [offset_cx2, offset_cy2], [w, h]], # 우, right
            [ [0,h], [offset_cx2, offset_cy2], [w, h]]] # 하, bottom

    
    for i in range(4):
        # 각각의 삼각형 좌표에 대해 어핀 변환 적용
        matrix = cv2.getAffineTransform( np.float32(tri1[i]), \
                                         np.float32(tri2[i]))
        warped = cv2.warpAffine( roi.copy(), matrix, (w, h), \
            None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
        # 삼각형 모양의 마스크 생성
        mask = np.zeros((h, w), dtype = np.uint8)
        cv2.fillConvexPoly(mask, np.int32(tri2[i]), (255,255,255))
        
        # 마스킹 후 합성
        warped = cv2.bitwise_and(warped, warped, mask=mask)
        out = cv2.bitwise_and(out, out, mask=cv2.bitwise_not(mask))
        out = out + warped

    # 관심 영역을 원본 영상에 합성
    img[y:y+h, x:x+w] = out
    return img 

# 마우스 이벤트 핸들 함수
def onMouse(event,x,y,flags,param):     
    global cx1, cy1, isDragging, img      # 전역변수 참조
    # 마우스 중심 점을 기준으로 대상 영역 따라다니기
    if event == cv2.EVENT_MOUSEMOVE:  
        if not isDragging :
            img_draw = img.copy()       
            # 드래그 영역 표시
            cv2.rectangle(img_draw, (x-half, y-half), \
                    (x+half, y+half), (0,255,0)) 
            cv2.imshow(win_title, img_draw) # 사각형 표시된 그림 화면 출력
    elif event == cv2.EVENT_LBUTTONDOWN :   
        isDragging = True                   # 드래그 시작
        cx1, cy1 = x, y                     # 드래그 시작된 원래의 위치 좌표 저장
    elif event == cv2.EVENT_LBUTTONUP :
        if isDragging:
            isDragging = False              # 드래그 끝
            # 드래그 시작 좌표와 끝난 좌표로 리퀴파이 적용 함수 호출
            liquify(img, cx1, cy1, x, y)    
            cv2.imshow(win_title, img)

if __name__ == '__main__' :
    img = cv2.imread("img/man_face.jpg")
    h, w = img.shape[:2]

    cv2.namedWindow(win_title)
    cv2.setMouseCallback(win_title, onMouse) 
    cv2.imshow(win_title, img)
    while True:
        key = cv2.waitKey(1)
        if key & 0xFF == 27:
            break
    cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
rows, cols = 240, 320
map_y, map_x = np.indices((rows, cols), dtype=np.float32)

# 거울 왜곡 효과 
map_mirrorh_x,map_mirrorh_y = map_x.copy(), map_y.copy() 
map_mirrorv_x,map_mirrorv_y = map_x.copy(), map_y.copy()    
## 좌우 대칭 거울 좌표 연산
map_mirrorh_x[: , cols//2:] = cols - map_mirrorh_x[:, cols//2:]-1
## 상하 대칭 거울 좌표 연산
map_mirrorv_y[rows//2:, :] = rows - map_mirrorv_y[rows//2:, :]-1

# 물결 효과
map_wave_x, map_wave_y = map_x.copy(), map_y.copy()
map_wave_x = map_wave_x + 15*np.sin(map_y/20)
map_wave_y = map_wave_y + 15*np.sin(map_x/20)    


# 렌즈 효과
## 렌즈 효과, 중심점 이동
map_lenz_x = 2*map_x/(cols-1)-1
map_lenz_y = 2*map_y/(rows-1)-1
## 렌즈 효과, 극좌표 변환
r, theta = cv2.cartToPolar(map_lenz_x, map_lenz_y)
r_convex = r.copy()
r_concave = r
## 볼록 렌즈 효과 매핑 좌표 연산
r_convex[r< 1] = r_convex[r<1] **2  
print(r.shape, r_convex[r<1].shape)
## 오목 렌즈 효과 매핑 좌표 연산
r_concave[r< 1] = r_concave[r<1] **0.5
## 렌즈 효과, 직교 좌표 복원
map_convex_x, map_convex_y = cv2.polarToCart(r_convex, theta)
map_concave_x, map_concave_y = cv2.polarToCart(r_concave, theta)
## 렌즈 효과, 좌상단 좌표 복원
map_convex_x = ((map_convex_x + 1)*cols-1)/2
map_convex_y = ((map_convex_y + 1)*rows-1)/2
map_concave_x = ((map_concave_x + 1)*cols-1)/2
map_concave_y = ((map_concave_y + 1)*rows-1)/2

while True:
    ret, frame = cap.read()
    # 준비한 매핑 좌표로 영상 효과 적용
    mirrorh=cv2.remap(frame,map_mirrorh_x,map_mirrorh_y,cv2.INTER_LINEAR)
    mirrorv=cv2.remap(frame,map_mirrorv_x,map_mirrorv_y,cv2.INTER_LINEAR)
    wave = cv2.remap(frame,map_wave_x,map_wave_y,cv2.INTER_LINEAR, \
                    None, cv2.BORDER_REPLICATE)
    convex = cv2.remap(frame,map_convex_x,map_convex_y,cv2.INTER_LINEAR)
    concave = cv2.remap(frame,map_concave_x,map_concave_y,cv2.INTER_LINEAR)
    # 영상 합치기
    r1 = np.hstack(( frame, mirrorh, mirrorv))
    r2 = np.hstack(( wave, convex, concave))
    merged = np.vstack((r1, r2))

    cv2.imshow('distorted',merged)
    if cv2.waitKey(1) & 0xFF== 27:
        break
cap.release
cv2.destroyAllWindows()

(240, 320) (59868,)
